# Saskatchewan Ag Export Processing (2024-2025)

任务目标：读取并清洗 CA-CN2024-2025report.csv，然后生成可视化图表。

执行步骤：
1. 读取与清洗
2. 数据质检
3. 可视化（Plotly）

遇到报错将自动修正，直到图表显示为止。

In [ ]:
# 安装/导入依赖
try:
    import pandas as pd
except Exception as e:
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "pandas"], check=False)
    import pandas as pd

try:
    import plotly.express as px
except Exception as e:
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "plotly"], check=False)
    import plotly.express as px

print("Dependencies ready:", pd.__version__)

In [ ]:
# 读取当前目录 CSV：优先尝试 your_data.csv，否则列出目录并定位 CA-CN2024-2025report.csv
import os
import glob

csv_path = None
candidates = ["your_data.csv", "SK-CN_2024-2025Oct_Report.csv"]
for name in candidates:
    if os.path.exists(name):
        csv_path = name
        break

if csv_path is None:
    # 列出目录所有文件，尝试寻找类似报告文件
    files = os.listdir('.')
    print("目录文件：", files)
    matches = [f for f in files if f.lower().endswith('.csv')]
    if len(matches) == 1:
        csv_path = matches[0]
    else:
        # 如果存在 SK-CN_2024-2025Oct_Report.csv 则用它
        for f in matches:
            if 'SK-CN_2024-2025Oct_Report.csv'.lower() in f.lower():
                csv_path = f
                break
        if csv_path is None and matches:
            csv_path = matches[0]

print("使用文件：", csv_path)
assert csv_path is not None, "未找到可用的 CSV 文件"

df = pd.read_csv(csv_path, skiprows=1)  # 跳过第1行
print("原始行数：", len(df))
df.head(2)

In [ ]:
# 列名与数据结构检查，辅助自动修正
print("列名：", list(df.columns))
print("数据类型：\n", df.dtypes)

# 打印文件前两行原始内容，便于判断分隔符与表头
with open(csv_path, 'r', encoding='utf-8', errors='ignore') as f:
    first_lines = [next(f).rstrip('\n') for _ in range(2)]
print("文件前两行：", first_lines)

In [ ]:
# 读取并清洗
import numpy as np

# 重新读取，跳过第一行的标题占位（如 \ufeffDomestic exports）
df = pd.read_csv(csv_path, skiprows=1)
# 清理可能的 BOM 与空格
df.columns = [c.replace('\ufeff', '').strip() for c in df.columns]

# 类型转换
df['Period'] = pd.to_datetime(df['Period'], errors='coerce')
df['Value ($)'] = pd.to_numeric(df['Value ($)'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')

# Commodity 拆分为 HS_Code 与 Name
split = df['Commodity'].astype(str).str.split(' - ', n=1, expand=True)
df['HS_Code'] = split[0]
df['Name'] = split[1]

# 过滤省份为 Saskatchewan
prov = df['Province'].fillna('').astype(str).str.strip().str.casefold()
df = df[prov == 'saskatchewan'].copy()

# 计算 Unit_Price（处理分母为0或空）
df['Unit_Price'] = np.where((df['Quantity'] > 0) & (~df['Quantity'].isna()), df['Value ($)'] / df['Quantity'], np.nan)

print('清洗完成。行数：', len(df))
df.head(3)

In [ ]:
# 数据质检：打印清洗后的前 3 行；检查 Unit of measure 是否统一为 'Weight in kilograms'
print("前 3 行：")
print(df.head(3))

uom_unique = df['Unit of measure'].dropna().unique()
print("Unit of measure 唯一值：", uom_unique)

if not (len(uom_unique) == 1 and uom_unique[0] == 'Weight in kilograms'):
    print("警告：Unit of measure 非全为 'Weight in kilograms' 或存在空值。")
else:
    print("Unit of measure 统一为 'Weight in kilograms'.")

In [ ]:
df_top = df.copy()
df_top["Commodity_Name"] = df_top["Name"]

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly.colors as pc
import numpy as np
import json
import base64

def map_broad(name):
    s = str(name).lower()
    if "barley" in s:
        return "Barley (All)"
    if "red spring wheat" in s:
        return "Wheat (Red Spring)"
    if "rape/colza" in s:
        return "Canola Complex"
    if "peas" in s:
        return "Peas (Yellow)"
    if "potash" in s or "potassium chloride" in s:
        return "Potash"
    if "wood pulp" in s:
        return "Wood pulp"
    if "soya bean" in s or "soya beans" in s or "soybean" in s or "soy beans" in s:
        return "Soya beans"
    return str(name)

df_top["Broad_Category"] = df_top["Commodity_Name"].apply(map_broad)

top_agg = df_top.groupby("Commodity_Name", as_index=False)["Value ($)"].sum()
top_agg = top_agg.sort_values("Value ($)", ascending=False).head(10)
top_list = top_agg["Commodity_Name"].tolist()

df_top = df_top[df_top["Commodity_Name"].isin(top_list)].copy()

if not np.issubdtype(df_top["Period"].dtype, np.datetime64):
    df_top["Period"] = pd.to_datetime(df_top["Period"]) 

df_top_grouped = df_top.groupby(["Period", "Commodity_Name", "Broad_Category"], as_index=False)["Value ($)"].sum()

df_total = df_top.groupby("Period", as_index=False)["Value ($)"].sum()

df_top_grouped["Period_str"] = df_top_grouped["Period"].dt.strftime('%Y-%m')
df_total["Period_str"] = df_total["Period"].dt.strftime('%Y-%m')
months = df_total.sort_values("Period")["Period_str"].tolist()

specific_color_map = {
    "Red spring wheat, o/t certified organic, grade 2, o/t seed for sowing": "#1f77b4",
    "Red spring wheat, o/t certified organic, grade 1, o/t seed for sowing": "#00BFFF",
    "Rape/colza seeds,low erucic acid, for oil extraction, w/n broken": "#d62728",
    "Rape/colza seed oil-cake & o solid residue, low erucic acid, w/n ground/pellet": "#D64327",
    "Barley, for malting, o/t seed for sowing": "#2ca02c",
    "Barley, o/t certified organic, o/t seed for sowing or malting": "#7CFC00",
    "Peas, yellow, nes, dried, shelled, w/n skinned": "#FECB52",
    "Potassium chloride, in packages weighing more than 10 kg": "#AB63FA",
    "Wood pulp, obtained by a combination of mechanical & chemical pulping processes": "#FFA15A",
    "Soya beans": "#19D3F3",
}

fig = go.Figure()

commodities_order = (
    df_top_grouped[["Commodity_Name"]].drop_duplicates()["Commodity_Name"].tolist()
)

for cname in commodities_order:
    sub = df_top_grouped[df_top_grouped["Commodity_Name"] == cname].sort_values("Period")
    x_list = sub["Period_str"].tolist()
    y_list = [float(v) if v is not None else None for v in sub["Value ($)"].tolist()]
    broad = sub["Broad_Category"].iloc[0]
    fig.add_trace(
        go.Bar(
            x=x_list,
            y=y_list,
            name=cname,
            marker=dict(color=specific_color_map.get(cname, "#00CED1")),
            legendgroup=broad,
            customdata=[cname]*len(x_list),
            hovertemplate="<b>%{x}</b><br>Commodity: %{customdata}<br>Value: $%{y:,.0f}<extra></extra>"
        )
    )

sub_total = df_total.sort_values("Period")
x_total = sub_total["Period_str"].tolist()
y_total = [float(v) if v is not None else None for v in sub_total["Value ($)"].tolist()]

fig.add_trace(
    go.Scatter(
        x=x_total,
        y=y_total,
        name="TOTAL Trend",
        mode="lines+markers",
        line=dict(color="white", width=3, dash="solid"),
        hovertemplate="<b>%{x}</b><br>TOTAL: $%{y:,.0f}<extra></extra>"
    )
)

names = [trace.name for trace in fig.data]
name_to_broad = (
    df_top_grouped.groupby("Commodity_Name")["Broad_Category"].first().to_dict()
)
broad_categories = sorted(list(df_top_grouped["Broad_Category"].unique()))

buttons = []

visible_overview = [True] * len(names)
buttons.append(
    dict(
        label="Overview (Stacked)",
        method="update",
        args=[{"visible": visible_overview}],
    )
)

for broad in broad_categories:
    visible = []
    for nm in names:
        if nm == "TOTAL Trend" or name_to_broad.get(nm) == broad:
            visible.append(True)
        else:
            visible.append(False)
    buttons.append(
        dict(
            label=broad,
            method="update",
            args=[{"visible": visible}],
        )
    )

fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0,
            xanchor="left",
            y=1.15,
            yanchor="top",
            bgcolor="rgba(0,0,0,0)",
            font=dict(color="white")
        )
    ],
    legend=dict(
        x=1.02,
        y=1,
        xanchor="left",
        yanchor="top",
    ),
    margin=dict(t=120, l=40, r=40, b=40),
    title="Saskatchewan Ag Export Composition (Monthly)",
    title_y=0.95,
    title_x=0.5,
    xaxis=dict(automargin=True),
    yaxis=dict(automargin=True),
    template="plotly_dark",
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    barmode="stack"
)

fig.update_xaxes(type='category', categoryorder='array', categoryarray=months)

fig.show()

fig_json = fig.to_json()
obj = json.loads(fig_json)

with open("export_data.json", "w", encoding="utf-8") as f:
    json.dump(obj, f)

with open("export_data.json", "r", encoding="utf-8") as f:
    obj = json.load(f)

def decode_array(value):
    if isinstance(value, dict) and "bdata" in value and "dtype" in value:
        arr = np.frombuffer(base64.b64decode(value["bdata"]), dtype=value["dtype"]) 
        return arr.tolist()
    return value

for trace in obj.get("data", []):
    if "x" in trace:
        trace["x"] = decode_array(trace["x"])
    if "y" in trace:
        trace["y"] = decode_array(trace["y"])

with open("export_data.json", "w", encoding="utf-8") as f:
    json.dump(obj, f)

line = json.dumps(obj)
idx = line.find('"y":')
if idx != -1:
    print(line[idx:idx + 160])